## 6. Riepilogo e Conclusioni

La matrice di correlazione mostra:

- **Correlazione tra `XGBoost_Proba` e `MLP_Proba`**: Il valore di correlazione tra le probabilità dei due modelli è il dato più importante.
  - Un valore **alto (es. > 0.85)** indica che i modelli tendono a fare previsioni molto simili. Concordano sulla maggior parte dei casi.
  - Un valore **moderato (es. 0.6 - 0.85)** suggerisce che, pur avendo una tendenza simile, ci sono aree di disaccordo significative. Questo è lo scenario ideale per un *ensemble*, poiché un modello potrebbe correggere gli errori dell'altro.
  - Un valore **basso (es. < 0.6)** indicherebbe che i modelli hanno imparato pattern quasi completamente diversi.

- **Correlazione con `TrueLabel`**: Entrambi i modelli dovrebbero mostrare una correlazione positiva con le etichette vere, indicando che le loro probabilità aumentano correttamente per la classe positiva. Il modello con la correlazione più alta rispetto a `TrueLabel` è, in generale, il più performante singolarmente.

L'analisi di questa matrice ci fornisce un'indicazione quantitativa sulla diversità dei nostri modelli e sulla potenziale efficacia di combinarli.

In [1]:
# Creazione DataFrame per l'analisi
predictions_df = pd.DataFrame({
    'TrueLabel': y_test,
    'XGBoost_Proba': xgb_preds_proba,
    'MLP_Proba': mlp_preds_proba
})

# Calcolo della matrice di correlazione
correlation_matrix = predictions_df.corr()

print("Matrice di Correlazione:")
print(correlation_matrix)

# Visualizzazione con heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(
    correlation_matrix,
    annot=True,
    cmap='coolwarm',
    fmt='.3f',
    linewidths=.5
)
plt.title('Matrice di Correlazione tra Previsioni dei Modelli', fontsize=14, fontweight='bold')
plt.show()

NameError: name 'pd' is not defined

## 5. Calcolo e Visualizzazione della Matrice di Correlazione

Creiamo un DataFrame con le etichette vere e le probabilità previste da entrambi i modelli, quindi calcoliamo la matrice di correlazione.

In [ ]:
# Previsioni XGBoost
print("Generazione previsioni XGBoost...")
xgb_preds_proba = xgb_model.predict_proba(X_test)[:, 1]

# Previsioni MLP
print("Generazione previsioni MLP...")

# 1. Standardizzazione dei dati (usando lo scaler fittato sui dati di train)
# Per farlo correttamente, carichiamo X_train solo per fittare lo scaler
X_train_for_scaler = pd.read_csv('../data/processed_v3_balanced/X_train.csv')
scaler = StandardScaler()
scaler.fit(X_train_for_scaler)
X_test_scaled = scaler.transform(X_test)

# 2. Conversione a tensore
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

# 3. Generazione previsioni
with torch.no_grad():
    mlp_outputs = mlp_model(X_test_tensor)
    mlp_preds_proba = torch.sigmoid(mlp_outputs).cpu().numpy().flatten()

print("\n✅ Previsioni generate per entrambi i modelli.")
print(f"Esempio probabilità XGBoost: {xgb_preds_proba[:5]}")
print(f"Esempio probabilità MLP:    {mlp_preds_proba[:5]}")

Generazione previsioni XGBoost...
Generazione previsioni MLP...


## 4. Generazione delle Previsioni

Ora generiamo le previsioni (probabilità) per entrambi i modelli sul dataset di test.

- Per **XGBoost**, usiamo `predict_proba`.
- Per **MLP**, dobbiamo prima standardizzare i dati (come nel training), convertirli in tensori e poi applicare la funzione sigmoide all'output.

In [4]:
class MLP(nn.Module):
    def __init__(self, input_dim, dropout=0.3):
        super(MLP, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(dropout),
            
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(dropout),
            
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(dropout),
            
            nn.Linear(32, 1)
        )
    
    def forward(self, x):
        return self.network(x)

# Inizializza il modello e carica i pesi
input_dim = X_test.shape[1]
mlp_model = MLP(input_dim, dropout=0.3).to(device)
mlp_model.load_state_dict(torch.load('../models/mlp_baseline/model_weights.pth', map_location=device))
mlp_model.eval()  # Imposta il modello in modalità valutazione

print("✅ Architettura MLP definita e pesi caricati.")
print(mlp_model)

✅ Architettura MLP definita e pesi caricati.
MLP(
  (network): Sequential(
    (0): Linear(in_features=43, out_features=128, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=64, out_features=32, bias=True)
    (9): ReLU()
    (10): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.3, inplace=False)
    (12): Linear(in_features=32, out_features=1, bias=True)
  )
)


## 3. Definizione Architettura MLP

Definiamo la classe del modello MLP. L'architettura deve essere **identica** a quella usata durante il training per poter caricare correttamente i pesi salvati.

In [3]:
# Caricamento del dataset di test
print("Caricamento dati di test...")
X_test = pd.read_csv('../data/processed_v3_balanced/X_test.csv')
y_test = pd.read_csv('../data/processed_v3_balanced/y_test.csv')['BinaryIncidentGrade']

# Caricamento del modello XGBoost v2
print("Caricamento modello XGBoost v2...")
xgb_model = xgb.XGBClassifier()
xgb_model.load_model('../models/xgboost_v2/model.json')

# Caricamento del modello MLP richiede prima la definizione dell'architettura
print("Modello MLP pronto per essere definito e caricato.")

print(f"\nDimensioni X_test: {X_test.shape}")
print(f"Dimensioni y_test: {y_test.shape}")

Caricamento dati di test...
Caricamento modello XGBoost v2...
Modello MLP pronto per essere definito e caricato.

Dimensioni X_test: (149537, 43)
Dimensioni y_test: (149537,)


## 2. Caricamento Dati e Modelli

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline
sns.set_style('whitegrid')

# Verifica disponibilità Metal (MPS) per PyTorch
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: mps


## 1. Setup e Import Librerie

# Analisi di Correlazione tra Modelli: XGBoost vs MLP

Questo notebook analizza la correlazione tra le previsioni di due dei nostri migliori modelli:
1.  **XGBoost v2**: Il nostro modello campione basato su gradient boosting.
2.  **MLP Standard**: Una rete neurale Multi-Layer Perceptron.

L'obiettivo è capire quanto le previsioni dei due modelli siano simili o dissimili. Una bassa correlazione potrebbe indicare che i modelli catturano pattern diversi nei dati, suggerendo che un loro *ensemble* potrebbe portare a performance superiori.